In [9]:
# Focal Length Calibration: fy=900, fx from 850 to 1300 with 25-step intervals
import os

def generate_fy_fx_calibration_images():
    """
    Generate and save images with fixed fy=900 and fx from 850 to 1300 (step 25)
    to test the effect of different fx values
    """
    # Create output directory
    output_dir = "fy900_fx_calibration_images"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Fixed fy value and varying fx values
    fy_fixed = 900
    fx_values = range(850, 1325, 25)  # 850 to 1300 with 25-step intervals
    
    print(f"Generating {len(fx_values)} calibration images...")
    print(f"Fixed fy: {fy_fixed}")
    print(f"Varying fx: {list(fx_values)}")
    
    for fx in fx_values:
        # Generate image
        image = create_projection_1280x720(fx, fy_fixed, flip_x=True, flip_y=False)
        
        # Save image with descriptive filename
        filename = f"calibration_fx{fx}_fy{fy_fixed}_1280x720.png"
        filepath = os.path.join(output_dir, filename)
        
        # Convert and save
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(filepath, image_bgr)
        
        print(f"Saved: {filename}")
    
    print(f"\n✅ All {len(fx_values)} calibration images saved in '{output_dir}/' folder")
    print("📁 Browse the images to see how fx affects the horizontal perspective!")
    
    return output_dir, list(fx_values)

# Generate the fy/fx calibration images
output_directory, generated_fx_values = generate_fy_fx_calibration_images()

Generating 19 calibration images...
Fixed fy: 900
Varying fx: [850, 875, 900, 925, 950, 975, 1000, 1025, 1050, 1075, 1100, 1125, 1150, 1175, 1200, 1225, 1250, 1275, 1300]
Saved: calibration_fx850_fy900_1280x720.png
Saved: calibration_fx850_fy900_1280x720.png
Saved: calibration_fx875_fy900_1280x720.png
Saved: calibration_fx875_fy900_1280x720.png
Saved: calibration_fx900_fy900_1280x720.png
Saved: calibration_fx900_fy900_1280x720.png
Saved: calibration_fx925_fy900_1280x720.png
Saved: calibration_fx925_fy900_1280x720.png
Saved: calibration_fx950_fy900_1280x720.png
Saved: calibration_fx950_fy900_1280x720.png
Saved: calibration_fx975_fy900_1280x720.png
Saved: calibration_fx975_fy900_1280x720.png
Saved: calibration_fx1000_fy900_1280x720.png
Saved: calibration_fx1000_fy900_1280x720.png
Saved: calibration_fx1025_fy900_1280x720.png
Saved: calibration_fx1025_fy900_1280x720.png
Saved: calibration_fx1050_fy900_1280x720.png
Saved: calibration_fx1050_fy900_1280x720.png
Saved: calibration_fx1075_fy900

# RealSense Camera Image Reconstruction

This notebook helps you reconstruct the original camera view from your 3D point cloud data to match **d435_Color.png**.

**Goal**: Find the optimal focal length parameters to recreate the original 1280×720 (16:9) camera image.

**Process**:
1. Load your point cloud data
2. Test different focal length values
3. Reconstruct the image with original camera dimensions
4. Find parameters that best match the original view

In [1]:
import open3d as o3d
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

# Load point cloud (using relative path from utilities folder)
pcd = o3d.io.read_point_cloud("../d435.ply")  # Path relative to utilities folder
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

if colors.shape[0] == 0:
    raise ValueError("Point cloud has no colors!")

print(f"Loaded point cloud with {len(points)} points")
print(f"Z value range: {points[:, 2].min()} to {points[:, 2].max()}")



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loaded point cloud with 81669 points
Z value range: -3.72266 to -0.21582


In [2]:
# Camera setup to match original d435_Color.png (1280x720, 16:9 aspect ratio)
img_width, img_height = 1280, 720  # Original camera dimensions
cx, cy = img_width // 2, img_height // 2

# Filter points in front of camera (adjust based on your coordinate system)
valid = points[:, 2] < 0  # Negative Z for RealSense camera
x, y, z = points[valid, 0], points[valid, 1], points[valid, 2]
colors_filtered = colors[valid]

print(f"Using {len(x)} points after filtering for negative Z")
print(f"Z range after filtering: {z.min()} to {z.max()}")
print(f"Target reconstruction: {img_width}x{img_height} (16:9 aspect ratio)")

Using 81669 points after filtering for negative Z
Z range after filtering: -3.72266 to -0.21582
Target reconstruction: 1280x720 (16:9 aspect ratio)


## How to Use the Tuned Focal Length

1. Find the values of `fx` and `fy` that produce the clearest projection of your asparagus stalks
2. Update your `main_maskfilter.py` with these values
3. If needed, adjust the X and Y flipping options as well
4. Re-run your point cloud projection to get more accurate results

### Note on RealSense Camera Calibration

For even better results, you can use the RealSense SDK to get the exact intrinsic parameters of your camera:

```python
import pyrealsense2 as rs

pipeline = rs.pipeline()
config = rs.config()
profile = pipeline.start(config)
intrinsics = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()

fx, fy = intrinsics.fx, intrinsics.fy
cx, cy = intrinsics.ppx, intrinsics.ppy
```

This will provide camera-specific calibration parameters rather than approximated values.

In [3]:
# Image Analysis: Find aspect ratio and metadata
import os
from PIL import Image
from PIL.ExifTags import TAGS
import json

def analyze_image(image_path):
    """
    Analyze image metadata, aspect ratio, and dimensions
    """
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return None
    
    try:
        # Open and analyze the image
        with Image.open(image_path) as img:
            width, height = img.size
            aspect_ratio = width / height
            mode = img.mode
            format_type = img.format
            
            print(f"=== Image Analysis: {os.path.basename(image_path)} ===")
            print(f"Dimensions: {width} x {height} pixels")
            print(f"Aspect Ratio: {aspect_ratio:.4f} ({width}:{height})")
            print(f"Color Mode: {mode}")
            print(f"Format: {format_type}")
            print(f"File Size: {os.path.getsize(image_path)} bytes")
            
            # Extract EXIF data if available
            exifdata = img.getexif()
            if exifdata:
                print("\n=== EXIF Metadata ===")
                for tag_id in exifdata:
                    tag = TAGS.get(tag_id, tag_id)
                    data = exifdata.get(tag_id)
                    if isinstance(data, bytes):
                        data = data.decode('utf-8', errors='ignore')
                    print(f"{tag}: {data}")
            else:
                print("No EXIF metadata found")
            
            return {
                'width': width,
                'height': height,
                'aspect_ratio': aspect_ratio,
                'mode': mode,
                'format': format_type,
                'file_size': os.path.getsize(image_path)
            }
    except Exception as e:
        print(f"Error analyzing image: {e}")
        return None

# Analyze key images in the project
image_paths = [
    "../d435_Color.png"  # Original camera image

]

image_metadata = {}
for path in image_paths:
    metadata = analyze_image(path)
    if metadata:
        image_metadata[path] = metadata
    print("-" * 50)

print("\n=== Summary ===")
for path, metadata in image_metadata.items():
    if metadata:
        print(f"{os.path.basename(path)}: {metadata['width']}x{metadata['height']} (AR: {metadata['aspect_ratio']:.3f})")

=== Image Analysis: d435_Color.png ===
Dimensions: 1280 x 720 pixels
Aspect Ratio: 1.7778 (1280:720)
Color Mode: RGB
Format: PNG
File Size: 1420517 bytes
No EXIF metadata found
--------------------------------------------------

=== Summary ===
d435_Color.png: 1280x720 (AR: 1.778)


In [5]:
# Interactive Focal Length Tuning for 1280x720 (16:9) reconstruction
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

def create_projection_1280x720(fx, fy, flip_x=True, flip_y=False):
    """
    Project 3D points to 1280x720 image using the given focal lengths
    """
    target_width, target_height = 1280, 720
    cx_new, cy_new = target_width // 2, target_height // 2
    
    # Perspective projection
    if flip_x:
        u = target_width - ((fx * x / z) + cx_new)
    else:
        u = (fx * x / z) + cx_new
        
    if flip_y:
        v = target_height - ((fy * y / z) + cy_new)
    else:
        v = (fy * y / z) + cy_new
        
    # Clip to image bounds
    u_img = np.clip(u, 0, target_width - 1).astype(int)
    v_img = np.clip(v, 0, target_height - 1).astype(int)
    
    # Create image
    image = np.zeros((target_height, target_width, 3), dtype=np.uint8)
    depth_buffer = np.full((target_height, target_width), -np.inf)
    
    # Project points
    for i in range(len(u_img)):
        ui, vi = u_img[i], v_img[i]
        zi = z[i]
        
        if zi > depth_buffer[vi, ui]:
            depth_buffer[vi, ui] = zi
            image[vi, ui] = (colors_filtered[i] * 255).astype(np.uint8)
    
    return image

# Interactive widget for focal length tuning
@interact(
    fx=FloatSlider(min=400, max=1500, step=10, value=850, description='fx:'),
    fy=FloatSlider(min=400, max=1500, step=10, value=850, description='fy:'),
    flip_x=widgets.Checkbox(value=True, description='Flip X'),
    flip_y=widgets.Checkbox(value=False, description='Flip Y')
)
def tune_focal_length(fx, fy, flip_x, flip_y):
    """Interactive focal length tuning for 1280x720 reconstruction"""
    image = create_projection_1280x720(fx, fy, flip_x, flip_y)
    
    plt.figure(figsize=(16, 9))  # 16:9 aspect ratio for display
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"1280x720 Reconstruction - fx={fx}, fy={fy}")
    plt.axis('off')
    plt.show()
    
    print(f"Current settings: fx={fx}, fy={fy}, flip_x={flip_x}, flip_y={flip_y}")
    return fx, fy

interactive(children=(FloatSlider(value=850.0, description='fx:', max=1500.0, min=400.0, step=10.0), FloatSlid…